In [187]:

import cv2
import numpy as np
from matplotlib import pyplot as plt

imgo = cv2.imread("NewImage-1.png", cv2.IMREAD_GRAYSCALE)
img = cv2.threshold(imgo,128,255,cv2.THRESH_BINARY)[1]
print(img)
print(img.shape)
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.resizeWindow('img', 600, 400)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [255 255 255 255 255   0   0 255   0   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255   0   0   0 255   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]]
(10, 10)


In [188]:
def split_img(img,block_h,block_w):
    blocks = []
    h , w = img.shape[:2]
    if h%block_h!=0:
        print("invalid block height")
        return 
    if w%block_w!=0:
        print("invalid block width")
        return
    num_rows = w//block_w
    num_cols = h//block_h
    for y in range(num_cols):
        for x in range(num_rows):
                left = x * block_w
                upper = y * block_h
                right = left + block_w
                lower = upper + block_h
                block = img[upper:lower, left:right]
                blocks.append(block)
    return blocks

In [189]:
def CAC_encode(img,block_h,block_w):    
    blocks = split_img(img, block_w, block_h)
    codes = ['0','11','10']
    code = []
    for block in blocks:
        if np.all(block == 0):
            code.append(codes[0])
        elif np.all(block == 255):
            code.append(codes[1])
        else:
            code.append(codes[2])
            block = block.tolist()
            for blk in block:
                for b in blk:
                    if b == 255:
                        code.append(1)   
                    else:
                        code.append(b)
    code ="".join(map(str,code))
   # print("Code length: ", len(code))
    
    CR = img.size/len(code)
    RD = 1 - 1/CR
   # print("Compression Ratio: " , CR , " \nRelative Data Redundancy: ", RD) 
    return  block_h ,block_w , img.shape[0], img.shape[1],  code , CR , RD

In [190]:
block_h ,block_w , height,width, code = CAC_encode(img,5,5)[:5]

In [191]:
def CAC_reblock(code,block_h,block_w,height,width):
    codes = ['0','11','10']
    blocks = []
    i = 0
    while i in range(len(code)) :
        if code[i] == codes[0]:
            blocks.append(np.zeros((block_h, block_w), dtype=np.uint8))
            i=i+1
        elif code[i:i+2] == codes[1]:
            blocks.append(np.ones((block_h, block_w), dtype=np.uint8) * 255)
            i = i+2
        elif code[i:i+2] == codes[2]: 
            i=i+2
            arr = np.zeros((block_h, block_w), dtype=np.uint8)
            for r in range(0,block_h):
                for c in range(0,block_w):
                    arr[r][c] = int(code[i])
                    i = i+1
            blocks.append(arr * 255)
    print(blocks)

In [192]:
def CAC_decode(code,block_h,block_w,height,width):
    img = np.ones((height, width), dtype=np.uint8)
    codes = ['0','11','10']
    blocks = []
    i = 0
    x = 0
    y = 0
    while i in range(len(code)):
        if x >= width:
            y = y + block_h 
            x = 0
        
        if code[i] == codes[0]:
            for r in range(0,block_h):
                for c in range(0,block_w):
                    img[r+y][c+x] = 0
            x = x + block_w
            i = i+1
            
        elif code[i:i+2] == codes[1]:
            i = i+2
            for r in range(0,block_h):
                for c in range(0,block_w):
                    img[r+y][c+x] = 255
            x= x + block_w
            
        elif code[i:i+2] == codes[2]: 
            i = i+2
            for r in range(0,block_h):
                for c in range(0,block_w):
                    img[r+y][c+x] = int(code[i]) *255
                    i = i+1
            x = x + block_w
    return img

In [193]:
decoded_img = CAC_decode(code,block_h,block_w,height,width)
print(decoded_img)

decoded_img = cv2.cvtColor(decoded_img, cv2.COLOR_GRAY2BGR)
cv2.namedWindow('decoded_img', cv2.WINDOW_NORMAL)
cv2.resizeWindow('decoded_img', 600, 400)
cv2.imshow('decoded_img', decoded_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [255 255 255 255 255   0   0 255   0   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255   0   0   0 255   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]]


In [194]:
#generate random valid block sizes 
def populate(h,w,n):
    pop=[]
    while len(pop) < n:
        block_h = np.random.randint(1,  h)
        block_w = np.random.randint(1,  w)
        if h% block_h == 0 and w% block_w == 0:
            pop.append([block_h,block_w])
    return pop

In [195]:

def fitness(img,population):
    fitness=[]
    for entry in population:
        print(entry)
        if img.shape[0] % entry[0] == 0 and img.shape[1] % entry[1] == 0:
            fitness.append(CAC_encode(img,entry[0],entry[1])[5])
        else:
            fitness.append(-1)
    
    pop , fit = zip(*sorted(zip(population, fitness), key=lambda x: x[1], reverse=True))
    return list(pop),list(fit)


In [196]:
def binarize(p_bits,q_bits,n):
    n1 = n[0]
    n2 = n[1]

    b1 = (bin(n1)[2:])
    p_zeros = p_bits -len(b1)
    p = '0' * p_zeros + b1
    b2 = (bin(n2)[2:])
    q_zeros = q_bits -len(b2)
    q = '0' * q_zeros + b2
    return p + q 

def decimate(p_bits,q_bits,b):
    n1 = str(b[:p_bits])
    n2 = str(b[p_bits:])
    p = int(n1,2)
    q = int(n2,2)
    return [p,q]    

In [197]:
def crossover(p1,p2):
    cp = np.random.randint(0,len(p1))
    child_1 = p1[:cp] + p2[cp:]
    child_2 = p2[:cp] + p1[cp:]
    return [child_1 , child_2]

In [198]:
def mutate(chromosome,chance):
    if np.random.random() <= chance:
        mp = np.random.randint(0,len(chromosome))
        chromosome = str(chromosome[:mp]) + str(1-int(chromosome[mp])) + str(chromosome[mp+1:])
    return chromosome
mutate('00101111',0.99)

'00111111'

In [204]:
def Genetic(img,sample_size,chance):
    height=img.shape[0]
    width=img.shape[1]
    p_bits = len(bin(height)[2:]) 
    q_bits = len(bin(width)[2:])
    population = populate(height,width,sample_size)
    fits = []
    population , fits = fitness(img,population)
    best_fit = 0
    best_size = None
    while best_fit != fits[0]:
        for i in range(0,int(sample_size/2),2):
            children = crossover(binarize(height,width,population[i]),binarize(p_bits,q_bits,population[i+1]))
            print (children)
            population.append(decimate(p_bits,q_bits,mutate(children[0],chance)))
            population.append(decimate(p_bits,q_bits,mutate(children[1],chance)))
        population, fits = fitness(img,population)
        population = population[:sample_size]
        fits = fits[:sample_size]
        if fits[0]>best_fit:
            best_fit = fits[0]
            best_size = population[0]
    print('Best size : ' + best_size , '\nBest CR : ' + best_fit)        
Genetic(img,8,0.4)

[1, 5]
[1, 5]
[2, 5]
[2, 1]
[2, 5]
[1, 1]
[1, 1]
[5, 2]
['00000101', '00100001010000000010']
['0000000010', '000101010000000101']
[5, 2]
[2, 5]
[2, 5]
[1, 5]
[1, 5]
[2, 1]
[1, 1]
[1, 1]
[0, 5]


ZeroDivisionError: integer modulo by zero

In [ ]:
pop = populate(img.shape[0],img.shape[1],5)
print(pop)
print(fitness(img,pop)[0])

[[5, 1], [2, 2], [5, 2], [1, 2], [5, 2]]
[5, 1]
Image size: 100
Code length:  45
Compression Ratio:  2.2222222222222223  
Relative Data Redundancy:  0.55
[2, 2]
Image size: 100
Code length:  86
Compression Ratio:  1.1627906976744187  
Relative Data Redundancy:  0.14
[5, 2]
Image size: 100
Code length:  48
Compression Ratio:  2.0833333333333335  
Relative Data Redundancy:  0.52
[1, 2]
Image size: 100
Code length:  116
Compression Ratio:  0.8620689655172413  
Relative Data Redundancy:  -0.16000000000000014
[5, 2]
Image size: 100
Code length:  48
Compression Ratio:  2.0833333333333335  
Relative Data Redundancy:  0.52
[[5, 1], [5, 2], [5, 2], [2, 2], [1, 2]]
